## SetFit

**SetFit**（https://huggingface.co/blog/zh/setfit） 是一个高效的少样本学习框架，它不需要提示（prompts）或标签映射器（verbalizers）就能对句子转换器（Sentence Transformers）进行微调。使用 **SetFit** 进行小样本文本分类的特点包括：

1. **无需提示或标签映射器**：与其他少样本微调技术不同，SetFit 不需要手工制作的提示或标签映射器来将示例转换成适合底层语言模型的格式。它通过直接从少量标记的文本示例生成丰富的嵌入（embeddings），从而省去了这一步骤。

2. **快速训练**：SetFit 不需要像T0或GPT-3这样的大型模型就能达到高准确率。因此，它在训练和推理时通常要快得多。

3. **多语言支持**：SetFit 可以与 Hub 上的任何句子转换器一起使用，这意味着您可以通过简单地微调多语言检查点来对多种语言的文本进行分类。

4. **样本效率高且对噪声鲁棒**：SetFit 在样本效率上显著优于标准微调，并且对噪声更加鲁棒。例如，即使每个类别只有8个标记的例子，SetFit 在客户评论（CR）情感数据集上也能与在完整训练集（3000个例子）上微调的RoBERTa Large相媲美。

5. **简单性和效率**：SetFit 旨在简单高效。它首先在少量标记的示例上微调句子转换器模型（通常每类8或16个），然后在从微调的句子转换器生成的嵌入上训练分类头。

In [1]:
from datasets import load_dataset
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
import torch

In [2]:
dataset = load_dataset('csv', data_files='data.csv', delimiter=',',  names=["label", "review"])
dataset

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
#每个类别随机采样8条数据作为训练集
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=8)
train_dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 17
})

In [5]:
eval_dataset = dataset["train"].select(range(100))
eval_dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 100
})

In [6]:
test_dataset = dataset["train"].select(range(100, 1000))
test_dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 900
})

In [ ]:
#model_body=SentenceTransformer("uer/sbert-base-chinese-nli")
#model_head=LogisticRegression(class_weight="balanced") 不平衡数据集
#model = SetFitModel(model_body=model_body,model_head=model_head)

In [7]:
#model = SetFitModel._from_pretrained(r"E:\code\model\bge-small-en-v1.5",)
model = SetFitModel._from_pretrained(r"E:\code\model\sbert-base-chinese-nli", device = torch.device("cuda"))
model.labels = ["正常短信", "垃圾短信"]
print(model)

No sentence-transformers model found with name E:\code\model\sbert-base-chinese-nli. Creating a new one with MEAN pooling.
model_head.pkl not found in E:\code\model\sbert-base-chinese-nli, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


SetFitModel(model_body=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_head=LogisticRegression(), multi_target_strategy=None, normalize_embeddings=False, labels=['正常短信', '垃圾短信'], model_card_data=SetFitModelCardData(language=None, license=None, tags=['setfit', 'sentence-transformers', 'text-classification', 'generated_from_setfit_trainer'], model_name='SetFit', model_id=None, dataset_name=None, dataset_id=None, dataset_revision=None, task_name=None, st_id=None, hyperparameters={}, eval_results_dict={}, eval_lines_list=[], metric_lines=[], widget=[], predict_example=None, label_example_list=[], tokenizer_warning=Fa

In [8]:
args = TrainingArguments(
    batch_size=4,
    num_epochs=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric="accuracy",
    column_mapping={"review": "text", "label": "label"}  # Map dataset columns to text/label expected by trainer
)

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [9]:
trainer.train()

***** Running training *****
  Num unique pairs = 160
  Batch size = 4
  Num epochs = 4
  Total optimization steps = 160


Epoch,Training Loss,Validation Loss,Embedding Loss,Rate
1,No log,No log,0.262200,0.000017
2,No log,No log,0.191300,0.000011
3,No log,No log,0.190000,0.000006
4,No log,No log,0.186300,0.000000


  0%|          | 0/1917 [00:00<?, ?it/s]

  0%|          | 0/1917 [00:00<?, ?it/s]

  0%|          | 0/1917 [00:00<?, ?it/s]

  0%|          | 0/1917 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 160.


In [11]:
trainer.evaluate(test_dataset)

Applying column mapping to the evaluation dataset
***** Running evaluation *****


{'accuracy': 0.91}

In [12]:
preds = model.predict(test_dataset['review'], use_labels = False)
#print(preds)

In [13]:
from sklearn import metrics
classify_report = metrics.classification_report(test_dataset['label'], preds, digits = 4) #分类报告 support测试集样本数
print(classify_report) 
confusion_matrix = metrics.confusion_matrix(preds, test_dataset['label']) #混淆矩阵
print(confusion_matrix) 

              precision    recall  f1-score   support

           0     0.9959    0.9036    0.9475       809
           1     0.5301    0.9670    0.6848        91

    accuracy                         0.9100       900
   macro avg     0.7630    0.9353    0.8162       900
weighted avg     0.9488    0.9100    0.9209       900

[[731   3]
 [ 78  88]]


In [14]:
preds = model.predict([
    "感谢致电杭州萧山全金釜韩国烧烤店，本店位于金城路xxx号。韩式烧烤等，价格实惠、欢迎惠顾【全金釜韩国烧烤店】",
    "2010年12月21日10时许，被告人王某趁武强县金音建筑工地工友张某不在宿舍之际，盗窃张某枕头下现金3500元。",
])
preds

array(['1', '0'], dtype='<U5')

In [ ]:
model.save_pretrained("setfit-8-shot")

In [ ]:
model = SetFitModel.from_pretrained("setfit-8-shot")

In [15]:
preds = model.predict([
    "感谢致电杭州萧山全金釜韩国烧烤店，本店位于金城路xxx号。韩式烧烤等，价格实惠、欢迎惠顾【全金釜韩国烧烤店】",
    "2010年12月21日10时许，被告人王某趁武强县金音建筑工地工友张某不在宿舍之际，盗窃张某枕头下现金3500元。",
])
preds

array(['1', '0'], dtype='<U5')